In [1]:
import os

os.chdir("../src")

In [10]:
import pandas as pd
from submission import parse_log_to_dataframe

# Assuming you have two dataframes df1 and df2 containing the classification outputs
# df1 = pd.read_csv('../artifacts/archive/results/classification.csv', header=None)
# df2 = pd.read_csv('../best057.csv', header=None)

df1 = parse_log_to_dataframe([['../subset7000/logs/gemma2_ml_gemma2_log.txt']], ['source1'])
df2 = parse_log_to_dataframe([['../subset7000/logs/gemma2_ml_mistralnemo_log.txt']], ['source2'])

df_dataset = pd.read_csv('../datasets/public_data/wi_dataset.csv')
df_labels = pd.read_csv('../datasets/public_data/wi_labels.csv')
df_dataset['id'] = df_dataset['id'].astype(str)
df_labels['code'] = df_labels['code'].astype(str)
df1 = df1.rename(columns={'Job Ad ID': 'id', 'ISCO Code': 'label'})
df2 = df2.rename(columns={'Job Ad ID': 'id', 'ISCO Code': 'label'})
df1 = df1.merge(df_dataset, on='id')
df2 = df2.merge(df_dataset, on='id')
df1 = df1.merge(df_labels, left_on='label', right_on='code', how='left')
df2 = df2.merge(df_labels, left_on='label', right_on='code', how='left')
# df1['id'] = df1['id'].astype(str)
# df2['id'] = df2['id'].astype(str)
# df1 = df1.merge(df1_logs, left_on='id', right_on='Job Ad ID', how='right')
# df2 = df2.merge(df2_logs, left_on='id', right_on='Job Ad ID', how='right')

100%|██████████| 7169/7169 [00:00<00:00, 12447.77it/s]


Source: source1 includes the following logs from ['../subset7000/logs/gemma2_ml_gemma2_log.txt']

DF shape: (7168, 6)

Number of unique Job Ad IDs: 7168

Number of unique ISCO Codes: 395

...

DF shape: (18560, 6)

Number of unique Job Ad IDs: 18560

Number of unique ISCO Codes: 496

Number of not null ISCO Codes: 16724

Number of duplicated ids: 0

In [11]:
df2 = df2[df2['id'].isin(df1['id'])]
df1 = df1[df1['id'].isin(df2['id'])]

In [23]:
import pandas as pd
import random
import openai
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import Layout

# Store evaluations in a list
evaluations = []
# Function to show random row from dataframes
def show_random_row(button):
    global random_index
    random_index = random.randint(0, len(df1) - 1)
    id = df1.iloc[random_index]['id']
    title = df1.iloc[random_index]['title']
    desc = df1.iloc[random_index]['description_x']
    output1 = df1.iloc[random_index]['label_x']
    output1_title = df1.iloc[random_index]['label_y']
    output1_log = df1.iloc[random_index]['logs']
    output2 = df2.iloc[random_index]['label_x']
    output2_title = df2.iloc[random_index]['label_y']
    output2_log = df2.iloc[random_index]['logs']

    
    display_id.value = f"{id}"
    display_title.value = f"{title}"
    display_description.value = f"{desc}"
    display_output1.value = f"{output1}"
    display_output1_title.value = f"{output1_title}"
    display_output1_log.value = f"{output1_log}"
    display_output2.value = f"{output2}"
    display_output2_title.value = f"{output2_title}"
    display_output2_log.value = f"{output2_log}"


# Function to handle evaluation
def evaluate(button):
    selected_option = evaluation_dropdown.value
    if selected_option:
        evaluation = {
            "id": df1.iloc[random_index]['id'],
            "Output 1": df1.iloc[random_index]['label_x'],
            "Output 2": df2.iloc[random_index]['label_x'],
            "Chosen Output": selected_option,
            "Chosen From": "df1" if selected_option == "Output 1" else "df2" if selected_option == "Output 2" else "OpenAI"
        }
        evaluations.append(evaluation)
        display_evaluation_status.value = f"Evaluation saved for row {random_index+1}"
    else:
        display_evaluation_status.value = "Please select an option to evaluate."

# Function to export evaluations to CSV
def export_evaluations(button):
    if evaluations:
        eval_df = pd.DataFrame(evaluations)
        eval_df.to_csv('evaluations.csv', index=False)
        display_export_status.value = "Evaluations exported to 'evaluations.csv'"
    else:
        display_export_status.value = "No evaluations to export."

# Widgets for display
display_id = widgets.Text(value='', description='Id:', style={'description_width': '100px'}, disabled=True, layout=Layout(width='90%'))
display_title = widgets.Text(value='', description='Title:', disabled=True, style={'description_width': '100px'}, layout=Layout(width='90%'))
display_description = widgets.Textarea(value='', description='Description:', style={'description_width': '100px'}, disabled=True, layout=Layout(width='90%', height='100px'))
display_output1 = widgets.Text(value='', description='Gemma Pred:', style={'description_width': '100px'}, disabled=True, layout=Layout(width='90%'))
display_output1_title = widgets.Text(value='', description='Gemma Title:', style={'description_width': '100px'}, disabled=True, layout=Layout(width='90%'))
display_output1_log = widgets.Textarea(value='', description='Gemma Logs:', style={'description_width': '100px'}, disabled=True, layout=Layout(width='90%', height='150px'))
display_output2 = widgets.Text(value='', description='Mistral Pred:', style={'description_width': '100px'}, disabled=True, layout=Layout(width='90%'))
display_output2_title = widgets.Text(value='', description='Mistral Title:', style={'description_width': '100px'}, disabled=True, layout=Layout(width='90%'))
display_output2_log = widgets.Textarea(value='', description='Mistral Logs:', style={'description_width': '100px'}, disabled=True, layout=Layout(width='90%', height='150px'))
display_evaluation_status = widgets.Label(value="")
display_export_status = widgets.Label(value="")

# Dropdown for manual evaluation
evaluation_dropdown = widgets.Dropdown(
    options=['Output 1', 'Output 2', 'OpenAI Output'],
    description='Rate:',
    style={'description_width': '100px'},
    layout=Layout(width='50%')
)

# Buttons with improved layout
switch_row_button = widgets.Button(description="Switch Row", button_style='primary', layout=Layout(width='200px'))
evaluate_button = widgets.Button(description="Save Evaluation", button_style='info', layout=Layout(width='200px'))
export_button = widgets.Button(description="Export Evaluations", button_style='warning', layout=Layout(width='200px'))

# Link buttons to functions
switch_row_button.on_click(show_random_row)
evaluate_button.on_click(evaluate)
export_button.on_click(export_evaluations)

# Organize the layout using HBox and VBox
buttons_row = widgets.HBox([switch_row_button, export_button],layout=Layout(justify_content='center'))
evaluation_row = widgets.HBox([evaluation_dropdown, evaluate_button])

# Final layout for the display
app_layout = widgets.VBox([
    display_id,
    display_title,
    display_description,
    display_output1,
    display_output1_title,
    display_output1_log,
    display_output2,
    display_output2_title,
    display_output2_log,
    evaluation_row,
    buttons_row,
    display_evaluation_status,
    display_export_status
])

# Display the app
display(app_layout)

# Initial display
show_random_row(None)
